<a href="https://colab.research.google.com/github/johanhoffman/methods-in-computational-science/blob/main/MICS_Iterative_Methods_YinengWang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative Methods**
**Yineng Wang**

# **Abstract**

This report implements four functions:

1. Jacobi iteration
2. Gauss-Seidel iteration
3. Newton's method for scalar nonlinear equation
4. Newton's method for vector nonlinear equation

#**About the code**

This report is written by Yineng Wang, based on Johan Hoffman's template.

In [1]:
# Copyright (C) 2020,2021 Johan Hoffman (jhoffman@kth.se)
# Copyright (C) 2021 Yineng Wang (yineng@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# **Set up environment**

To have access to the neccessary modules you have to run this cell.

In [2]:
import time
import unittest

import numpy as np
from numpy.linalg import norm

# **Introduction**

Fixed point iteration is a performant method to solve equations. For a linear system of equations $Ax = b$, preconditioned Richardson iteration can be emplyoed, which takes the form

$$x^{(k+1)} = (I - \alpha B A) x^{(k)} + \alpha B b$$

or

$$x^{(k+1)} = (I - \alpha A B) x^{(k)} + \alpha b$$

By taking $\alpha = 1$ and $B = D^{-1}$ in left preconditioned Richardson iteration, we obtain Jacobi iteration

$$x^{(k+1)} = (I - D^{-1} A) x^{(k)} + D^{-1} b,$$

where $D$ is a diagonal matrix whose entries are the diagonal elements of $A$. And by taking $\alpha = 1$ and $B = L^{-1}$ in left preconditioned Richardson iteration, we obtain Gauss-Seidel iteration

$$x^{(k+1)} = (I - L^{-1} A) x^{(k)} + L^{-1} b,$$

where $L$ is a upper triangular matrix obtained from the matrix $A$ by zeroing out all entries above the diagonal.

Jacobi iteration and Gauss-Seidel iteration are especially efficient when $A$ is diagonal dominant. There are more constrains on the design of Gauss-Seidel iteration than Jacobi iteration.


If the equation $f(x) = 0$ is non-linear, one can apply Newton's method to solve it, which takes the form

$$x^{(k+1)} = x^{(k)} - f'(x^{(k)})^{-1} f(x^{(k)}).$$

It can be shown that Newton's method has a quadratic order of convergence.

# **Method**

The implementations are based on the lecture notes. NumPy's arrays are used to represent objects (vectors, matrices) of linear algebra.

## 1. Jacobi iteration for Ax=b

Input: matrix A, vector b

Output: vector x

Jacobi iteration is a stationary iterative method taking the form of
$x^{(k+1)} = (I - D^{-1} A) x^{(k)} + D^{-1} b$, where $D$ is a diagonal matrix whose entries are the diagonal elements of $A$. In terms of the components of the matrix $A=(a_{ij})$, we have

$$
\begin{align}
x_i^{(k+1)} &= (I - D^{-1} A) x^{(k)} + D^{-1} b \\
&= x_i^{(k)} - a_{ii}^{-1} \sum_{j=1}^{n} a_{ij} x_j^{(k)} + a_{ii}^{-1} b_i \\
&= a_{ii}^{-1} \left(b_i - \sum_{j\neq i} a_{ij} x_j^{(k)}\right), \forall i.
\end{align}
$$

In the implementation, we make use of NumPy's vectorized operators to avoid loops. The stopping criteria is based on the formula $\Vert r^{(k)} \Vert / \Vert b \Vert < TOL$ in section (7.3) of the lecture notes. We do not formally check whether the iteration converges (for example, by using the strictly diagonally dominant condition), but instead set a upper bound for the number of iterations, `max_iter`.

In [3]:
def jacobi_iteration(A, b, tol=1e-9, max_iter=2000):
    n = b.shape[0]
    d = A.diagonal()
    if np.any(d == 0):
        raise ValueError('there is a zero diagonal entry')
    D_inv_A = A / d.reshape((n,1))    # Equivalent to D^-1 A
    D_inv_b = b / d    # Equivalent to D^-1 b
    norm_b = norm(b)

    x = np.zeros((n,))
    n_iter = 0
    while True:
        residual = A @ x - b
        if norm(residual) / norm_b < tol:
            return x
        x = x - D_inv_A @ x + D_inv_b    # Equivalent to x = (I - D^-1 A) x + D^-1 b
        n_iter += 1
        if n_iter > max_iter:
            raise ValueError('the iteration diverges')

## 2. Gauss-Seidel iteration for Ax=b

Input: matrix A, vector b

Output: vector x

Gauss-Seidel iteration is a stationary iterative method taking the form of
$x^{(k+1)} = (I - L^{-1} A) x^{(k)} + L^{-1} b$, where $L$ is a upper triangular matrix obtained from the matrix $A$ by zeroing out all entries above the diagonal.

Since $x^{(k+1)} = (I - L^{-1} A) x^{(k)} + L^{-1} b = x^{(k)} + L^{-1}(A x^{(k)} + b)$ and $L$ is upper triangular, we can solve get the value of $L^{-1}(-A x^{(k)} + b)$ by forward substitution (see algorithm (5.1) of the lecture notes). Denote $L^{-1}(-A x^{(k)} + b)$ by $y^{(k)}$, then we have

$$
y_i^{(k)} = a_{ii}^{-1}\left(b_i - \sum_{j=1}^n a_{ij} x_j^{(k)} - \sum_{j<i} a_{ij} y_j^{(k)}\right),
$$

and

$$
\begin{align}
x_i^{(k+1)} &= x_i^{(k)} + y_i^{(k)} \\
&= x_i^{(k)} + a_{ii}^{-1}\left(b_i - \sum_{j=1}^n a_{ij} x_j^{(k)} - \sum_{j<i} a_{ij} y_j^{(k)}\right) \\
&= a_{ii}^{-1} a_{ii} x_i^{(k)} + a_{ii}^{-1}\left(b_i - \sum_{j=1}^n a_{ij} x_j^{(k)} - \sum_{j<i} a_{ij} (x_j^{(k+1)} - x_j^{(k)})\right) \\
&= a_{ii}^{-1} \left(b_i - \sum_{j>i} a_{ij} x_j^{(k)} - \sum_{j<i} a_{ij} x_j^{(k+1)}\right), \forall i.
\end{align}
$$

Unlike Jacobi iteration, Gauss-Seidel iteration cannot be implemented by taking considerable advantage of NumPy's vectorization, but it's still worth a try.

In [4]:
def gauss_seidel_iteration(A, b, tol=1e-9, max_iter=2000):
    n = b.shape[0]
    d = A.diagonal()
    if np.any(d == 0):
        raise ValueError('there is a zero diagonal entry')
    norm_b = norm(b)

    x = np.zeros((n,))
    n_iter = 0
    while True:
        residual = A @ x - b
        if norm(residual) / norm_b < tol:
            return x
        for i in range(n):
            x[i] = (b[i] - A[i,:i] @ x[:i] - A[i,(i+1):] @ x[(i+1):]) / d[i]
        n_iter += 1
        if n_iter > max_iter:
            raise ValueError('the iteration diverges')

## 3. Newton's method for scalar nonlinear equation f(x)=0

Input: scalar function f(x)

Output: real number x

For the equation $f(x) = 0$, Newton's method takes the form

$$x^{(k+1)} = x^{(k)} - \frac{f(x^{(k)})}{f'(x^{(k)})}.$$

To get the derivative of $f(x)$, we use a finite difference approximation. 

In [5]:
def derivative(f, x, h):
    return (f(x + h/2) - f(x - h/2)) / h


def newton_method(f, tol=1e-9, max_iter=2000, x0=0, h=1e-6):
    x = x0
    n_iter = 0
    while True:
        f_x = f(x)
        if abs(f_x) < tol:
            return x
        x -= f_x / derivative(f, x, h)
        n_iter += 1
        if n_iter > max_iter:
            raise ValueError('the iteration diverges')

## 4. Newton's method for vector nonlinear equation f(x)=0

Input: vector function f(x)

Output: vector x

Newton's method for the vector nonlinear equation $f(x) = 0, f: R^n \rightarrow R^n$ takes the form

$$
x^{(k+1)} = x^{(k)} - f'(x^{(k)})^{-1} f(x^{(k)}),
$$
where $f'(x)$ is the Jacobian matrix of $f$ at $x$.

The construction of the inverse of $f'(x)$ is costly, but can be approximated by solving the linear system of equations $f'(x^{(k)}) \Delta x = -f(x^{(k)})$. In the implementation, finite difference approximation is used to compute the Jacobian matrix, and Gauss-Seidel iteration is employed to solve the linear system.

In the implementation, besides `f`, another parameter `x0: np.array` is used to indicate $f$'s domain, as well as work as the initial guess of `x`.

In [6]:
def jacobian(f, x, m, n, h):
    J = np.zeros((m, n))
    for j in range(n):
        dx = np.zeros(n)
        dx[j] = h/2
        Df = (f(x+dx) - f(x-dx)) / h
        J[:,j] = Df
    return J


def newton_method_vector(f, x0, tol=1e-9, max_iter=2000, h=1e-6):
    x = x0.copy()
    n_iter = 0
    while True:
        f_x = f(x)
        if norm(f_x) < tol:
            return x
        J = jacobian(f, x, f_x.shape[0], x.shape[0], h)
        x -= gauss_seidel_iteration(J, f_x)
        n_iter += 1
        if n_iter > max_iter:
            raise ValueError('the iteration diverges')

# **Results**

In this section, we test the implementations of the four functions.
<!-- The exact solutions of equations are obtained by using NumPy's solver. -->

## 1. Jacobi iteration for Ax=b

We test the convergence of residual $\Vert Ax-b \Vert$, $\Vert x-y \Vert$ for manufactured/exact solution $y$.

In [7]:
class TestJacobiIteration(unittest.TestCase):
    def test_correctness(self):
        A = np.array(
            [[2,1,0],
             [-1,3,-1],
             [0,1,2]])
        b = np.array([4,-10,0])
        x = jacobi_iteration(A, b)
        y = np.array([3,-2,1])
        self.assertTrue(norm(A@x - b) / norm(b) < 1e-9)
        np.testing.assert_almost_equal(x, y)

## 2. Gauss-Seidel iteration for Ax=b

We test the convergence of residual $\Vert Ax-b \Vert$, $\Vert x-y \Vert$ for manufactured/exact solution $y$.

In [8]:
class TestGaussSeidelIteration(unittest.TestCase):
    def test_correctness(self):
        A = np.array(
            [[3,1,-1],
             [2,-5,2],
             [1,6,8]])
        b = np.array([1,9,3])
        x = gauss_seidel_iteration(A, b)
        y = np.array([1,-1,1])
        self.assertTrue(norm(A@x - b) / norm(b) < 1e-9)
        np.testing.assert_almost_equal(x, y)

## 3. Newton's method for scalar nonlinear equation f(x)=0

We test the convergence of residual $|f(x)|$, $|x-y|$ for manufactured/exact solution $y$.

In [9]:
class TestNewtonMethodScalar(unittest.TestCase):
    def test_correcness(self):
        def f(x):
            return (x+1) * (x-1) * (x-3)
        x = newton_method(f, x0=3.5)
        y = 3
        self.assertTrue(abs(f(x)) < 1e-9)
        self.assertAlmostEqual(x, y)

## 4. Newton's method for vector nonlinear equation f(x)=0

We test the convergence of residual $\Vert f(x) \Vert$, $\Vert x-y \Vert$ for manufactured/exact solution $y$.

In [10]:
class TestNewtonMethodVector(unittest.TestCase):
    def test_correcness(self):
        def f(x):
            return (x+1) * (x-1) * (x-3)
        x = newton_method_vector(f, x0=np.array([-1.25, 1.25, 3.5]))
        y = np.array([-1.0, 1.0, 3.0])
        self.assertTrue(norm(f(x)) < 1e-9)
        np.testing.assert_almost_equal(x, y)

## Conduct the tests

In [11]:
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_correctness (__main__.TestGaussSeidelIteration) ... ok
test_correctness (__main__.TestJacobiIteration) ... ok
test_correcness (__main__.TestNewtonMethodScalar) ... ok
test_correcness (__main__.TestNewtonMethodVector) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.061s

OK


# **Discussion**

The test results are as expected. It is worth a exploratory study to investigate the conditions under which Jacobi iteration and Gauss-Seidel iteration converge. The convergence of Newton's method depends on how close the initial guess of the solution is to the exact solution, thus requiring a relatively accurate guess.